In [1]:
# Helper function for division with remainder
def divisionWithRemainder(a,b):
    return a // b, a % b

In [2]:
def extendedGCD(a,b):
    a, b = abs(a), abs(b) # Make sure a and b are positive
    
    r0, r1 = a, b # Set r0 = a and r1 = b
    u0, u1 = 1, 0 # Coefficients for a (u0 = 1, u1 = 0 for the first step)
    v0, v1 = 0, 1 # Coefficients for b (v0 = 0, v1 = 1 for the first step)
    
    while r1 != 0:
        # Perform division with remainder
        q, r2 = divisionWithRemainder(r0, r1)
        
        # Update r0, r1 for the next iteration
        r0, r1 = r1, r2
        
        # Update u0, u1 based on the division
        u0, u1 = u1, u0 - q * u1
        
        # Update v0, v1 based on the division
        v0, v1 = v1, v0 - q * v1
        
    # At the end, r0 contains the gcd, u0 and v0 are the coefficients for a, b
    return r0, u0, v0

In [27]:
def modInverse(a, p):
    # Function to compute the modular inverse of a modulo p
    gcd, x, y = extendedGCD(a, p)
    if gcd != 1:
        raise ZeroDivisionError(a)
    return x % p

In [4]:
def addPoints(P, Q, E, p):
    # Extract curve parameters
    A, B = E
    
    # Check if either P or Q is point at infinity:
    if P == "O":
        return Q
    
    if Q == "O":
        return P
    
    x1, y1 = P
    x2, y2 = Q
    
    # Point doubling
    if P == Q:
        # If y1 is 0, the point is at infinity
        if y1 == 0:
            return "O"
        
        # Get slope for point doubling
        num = (3 * x1**2 + A) % p
        den = (2 * y1) % p
        
        # Find modular inverse of denominator 
        inv = modInverse(den, p)
        
        # Slope
        lam = (num * inv) % p
        
        # Compute x3 and y3
        x3 = (lam**2 - 2 * x1) % p
        y3 = (lam * (x1 - x3) - y1) % p
        return (x3, y3)
        
    # General case for adding 2 distinct primes
    # If x1 == x2, then P+Q is O
    if x1 == x2:
        return "O"
    
    # Slope for distinct primes
    num = (y2 - y1) % p
    den = (x2 - x1) % p

    # Find modular inverse of denominator
    inv = modInverse(den, p)
    
    # Slope
    lam = (num * inv) % p
    
    # Compute x3 and y3
    x3 = (lam**2 - x1 - x2) % p
    y3 = (lam * (x1 - x3) - y1) % p
    
    return (x3, y3)

In [5]:
def doubleAndAdd(P, n, E, p):      
    # Initialize the result as the point at infinity "O"
    R = "O"
    
    # Start with Q = P (the point we want to multiply by n)
    Q = P
    
    # While there are still bits left in the binary representation of n
    while n > 0:
        # If the current bit of n is 1, we add Q to R
        if n % 2 == 1:
            # Add R to Q
            R = addPoints(R, Q, E, p)
        # Double the point Q (this corresponds to squaring in fast exponentiation)
        Q = addPoints(Q, Q, E, p)
        
        # Shift n to the right by 1 (divide n by 2)
        n = n // 2
        
    # Return final result (nP)
    return R  

In [6]:
def fastPower(g, A, N):
    # If A is negative, compute the modular inverse of g^(-A)
    if A < 0:
        g = modInverse(g, N) # Invert the base g before raising it to -A
        
        A = -A # Now work with the positive equivalent of -A
        
    # Initialize a = g and b = 1
    g = g % N # Reduce g mod N first
    b = 1
    
    # Apply the algorithm while A > 0
    while A > 0:
        # If A is odd, multiply by a and take modulo N
        if A % 2 == 1:
            b = (b * g) % N
            
        # Square a and reduce A by half
        g = (g * g) % N
        A = A // 2
   
    # Return the result b
    return b 

In [7]:
def intToText(n):
    # Initialize an empty list to store the chars
    chars = []
    
    # Keep dividing n by 256 and collect remainders
    while n > 0:
        # Get remainder, this gives us a digit in base-256
        remainder = n % 256
        
        # Convert current digit into corresponding character using chr
        chars.append(chr(remainder))
        
        # Update n by didviding it by 256 for the next digit
        n = n // 256
        
    # Join together and return the list of chars at the end
    return ''.join(chars)

In [8]:
def textToInt(w):
    # Initialize n, this will hold final int result
    n = 0
    # Loop over each character in string w, along with its index i
    for i, char in enumerate(w):
        # Get the ascii value of current character using ord(char)
        # Multiply it by 256^i to get its value in base 256
        n += ord(char)*(256**i)
    # After processing all the characters, return the accumulated int n    
    return n

In [9]:
import random
def findPrime(lowerBound, upperBound):
    # Keep trying until we find a prime
    while True:
        # Pick a random number between lowerBound and upperBound
        n = random.randint(lowerBound, upperBound)
        
        # Check if the number is probably prime
        if probablyPrime(n):
            return n # Return the prime number found 'n'

In [10]:
import random
def probablyPrime(n):
    # Run the MillerRabin test 20 times with random base 'a'
    for _ in range(20):
        a = random.randint(2, n-1) # Random int between 2 and n-1
        # Check if current a is a MillerRabin witness for 'n'
        if MillerRabin(a, n):
            return False # If we find a witness, n is composite
    return True # If no witness is found after 20 tests, n is probably prime

In [11]:
def MillerRabin(a, n):
    # Factor n-1 into 2^k * m, where m is odd
    k = 0
    m = n-1
    while m % 2 == 0:
        m//= 2
        k += 1
        
    # Compute a^m mod n using fast exponentiation
    am = fastPower(a, m, n)
    
    # If a^m ≡ 1 mod n, then a is not a witness
    if am == 1:
        return False
    
    # Check for a^2^i * m ≡ -1 mod n for i = 0, ..., k-1
    for i in range(k):
        if am == n - 1:
            return False
        am = (am * am) % n # Compute a^2^i by succesive squaring
        
    # If we never found a^2^i * m ≡ -1 mod n, then a is a witness
    return True

In [12]:
def ElGamalParameterCreation(b):
    # Generate a b-bit prime number p
    lowerBound = 2**(b-1)
    upperBound = 2**b-1
    
    # Find a generator g for Z_p^*
    # Retry until we find a p s.t. p = 2q + 1 where q is also prime
    while True:
        p = findPrime(lowerBound, upperBound)
        q = (p-1)//2
        if probablyPrime(q):
            break
        
    # Try to find a generator g such that g^((p-1)/2) !≡ 1 mod p or g^1 !≡ 1 mod p
    while True:
        g = random.randint(2, p-2)
        if fastPower(g, 2, p) != 1 and fastPower(g,q,p) != 1:
            break
            
    return [p, g]

In [13]:
def ElGamalKeyCreation(pubParams):
    p, g = pubParams
    
    # Private key, random s  ∈ {1, ..., p-2}
    s = random.randint(1, p-2)
    
    # Compute g^s mod p
    gs = fastPower(g, s, p)
    
    #Public key 
    v = fastPower(g, s, p)
    
    return s, v

In [14]:
def ElGamalSign(pubParams, signingKey, document):
    p,g = pubParams
    s = signingKey
    
    # Convert document to integer hash modulo (p-1)
    h = textToInt(document) % (p - 1)
    
    while True:
        # Pick random beta ∈ Z_{p-1}^*
        beta = random.randint(1, p-2)
        if extendedGCD(beta, p-1)[0] == 1:
            break
            
            
    # Compute S1 = g^beta mod p
    S1 = fastPower(g, beta, p)
    
    # Compute S2 = (h - s * S1) * beta^-1 mod (p-1)
    beta_inv = modInverse(beta, p-1)
    S2 = (h - s * S1) * beta_inv % (p - 1)
    
    print(f"Signature: S1 = {S1}, S2 = {S2}, beta_inv = {beta_inv}")
    
    return S1, S2


In [15]:
def ElGamalVerify(pubParams, verificationKey, document, signedDocument):
    p, g = pubParams
    v = verificationKey
    r, t = signedDocument
    
    # Verify r is in valid range
    if not (1 <= r < p):
        return False
    
    # Convert document to integer hash modulo (p-1)
    h = textToInt(document) % (p - 1)
    
    # Compute g^h mod p
    left = fastPower(g, h, p)
    
    # Compute v^r mod p
    vr = fastPower(v, r, p)
    
    # Compute r^t mod p
    rt = fastPower(r, t, p)
    
    # Compute v^r * r^t mod p
    right = (vr * rt) % p
    
    # Check if both sides match
    return left == right

In [ ]:
# Testing
# Generate a 32-bit prime
pubParams = ElGamalParameterCreation(32)
p,g = pubParams
print("Public parameters:")
print("p = ", p)
print("g = ", g)

# Generate key pair
s,v = ElGamalKeyCreation(pubParams)
print("\nKeys:")
print("Signing key s =", s)
print("Verification key v =", v)

# Define document to sign
D = 314159
doc = intToText(D)

# Sign document
signature = ElGamalSign(pubParams, s, doc)
print("\nSignature (r, t): ", signature)

# Verify the signature
isValid = ElGamalVerify(pubParams, v, doc, signature)
print("\nVerification result: ", isValid)

Public parameters:
p =  4246397879
g =  4119787799

Keys:
Signing key s = 2909510369
Verification key v = 2360275797
Signature: S1 = 1487986737, S2 = 2213265780, beta_inv = 3537223589

Signature (r, t):  (1487986737, 2213265780)

Verification result:  True


In [ ]:
# Testing
# Question 2 documents
d = 320322645469855149716072227682460508269721725760557985886277190655697219
dPrime = 74549592591553434253650896145554141457666307549831202799378755

# Convert imtegers back to text
advice1 = intToText(d)
advice2 = intToText(dPrime)

# Print results
print("Avice 1: ", advice1)
print("Advice 2: ", advice2)

# Here are the public parameters
p = 63763036770305675425223818535163544823646748646012308847515047342847475899751
g = 20119259641437061188845699514503278028142305695487543873085942722254747288394

# Here is the public verification key
v = 33539746038778540759735362312361055037370039940261352677772334533434654814579

# Here are the signed documents
dSig = [29433470486560334144623951556645859140900248185071358201234156282753691310077,12860016104300249307248383217107923493203148899495279416185499586534166303722]

dPrimeSig = [13416511489189463535556426225612058855078916511050537218014145402736162121928,59359627103338619156875624611277097474231592296570819984905657071079102999679]

# Test verification of first document
valid_d = ElGamalVerify([p,g], v, advice1, dSig)
print(f"Verification result for the first document: {valid_d}")

# Test verification of second document
valid_d_prime = ElGamalVerify([p,g], v, advice2, dPrimeSig)
print(f"Verification result for the first document: {valid_d_prime}")

Avice 1:  Cats can have a little salami.
Advice 2:  Cats can only eat catfood.
Verification result for the first document: True
Verification result for the first document: False


In [18]:
def ECDSAKeyCreation(pubParams):
    E, p, G, q = pubParams
    
    # Generate a secret key s
    s = random.randint(1, q-1)
    
    # Compute the public key V = s * G using double and add
    V = doubleAndAdd(G, s, E, p)
    
    # Return private and public key
    return s, V

In [19]:
def ECDSASign(pubParams, signingKey, document):
    E, p, G, q = pubParams
    
    # Choose a random number e
    e = random.randint(1, q-1)
    
    # Compute eG (elliptic curve multiplication)
    eG = doubleAndAdd(G, e, E, p)
    
    # Compute s1 = x(eG) mod q
    s1 = eG[0] % q
    
    # Compute s2 = (d + s * s1) * e^-1 mod q
    d = document % q
    e_inv = modInverse(e, q)
    s2 = (d + signingKey * s1) * e_inv % q
    
    # Return the signature (s1, s2)
    return (s1, s2)

In [20]:
def ECDSAVerify(pubParams, verificationKey, document, signedDocument):
    E, p, G, q = pubParams
    
    # Extract signature
    s1, s2 = signedDocument
    
    # Check if s1 is in the valid range
    if not (1 <= s1 < q):
        return False
    
    # Compute the document hash d (mod q)
    d = document % q
    
    # Compute v1 = = d * s2^-1 mod q and v2 = s1 * s2^-1 mod q
    s2_inv = modInverse(s2, q)
    v1 = d * s2_inv % q
    v2 = s1 * s2_inv % q
    
    # Compute v1 * G + v2 * V
    v1G = doubleAndAdd(G, v1, E, p)
    v2G = doubleAndAdd(verificationKey, v2, E, p)
    resultPoint = addPoints(v1G, v2G, E, p)
    
    # Check if x(v1G + v2V) mod q == s1
    if resultPoint[0] % q == s1:
        return True
    else:
        return False

In [ ]:
# Testing
# Bitcoin curve parameters
A = 0
B = 7
E = (A, B)

p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

G = [55066263022277343669578718895168534326250603453777594175500187360389116729240,
     32670510020758816978083085130507043184471273380659243275938904335757337482424]

q = 115792089237316195423570985008687907852837564279074904382605163141518161494337

# Public parameters tuple
pubParams = (E, p, G, q)

# Generate key pair
s, V = ECDSAKeyCreation(pubParams)
print("Private signing key (s):", s)
print("Public verification key (V):", V)

# Document to sign
D = 314159

# Sign the document
sig = ECDSASign(pubParams, s, D)
print("Signature (s1, s2):", sig)

# Verify the signature
valid = ECDSAVerify(pubParams, V, D, sig)
print("Is signature valid?", valid)

Private signing key (s): 24230577249032040836786025054793116565973955441593696733158287960366667976753
Public verification key (V): (98263382152009145481561384084640717389398964188011115553575305078131909301488, 45108154843854704114633904968542912668745618338795873226259991965716171358125)
Signature (s1, s2): (107785601354580493682498245868474174301616056022346101679833869191483330933447, 59658342004526137249941407164815330001753986254619553113308117616392842747121)
Is signature valid? True


In [ ]:
# Testing
# Here are the parameters, keys, and documents for Question 4

# Here are the bitcoin parameters
A = 0
B = 7
E = (A, B)

#prime
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

#basepoint
G = [55066263022277343669578718895168534326250603453777594175500187360389116729240,32670510020758816978083085130507043184471273380659243275938904335757337482424]

#order of basepoint
q = 115792089237316195423570985008687907852837564279074904382605163141518161494337

#Here is my public verification key

V = [67545929220337950307684201000770305786305857763801479070770667679633981677471,57404668796908865765193875258816622070375393753567561160910848438073796688949]

#Here are the 2 documents
d = 19084714191624028315430109972252645018129944214203497594232383305
dPrime = 13749661396179088281771254246593875257372241209246875366837329993

#Here are the associated signed documents
dSig = [7821197833556217190755673939417527809613808935070956696104467705513893731806,52322483224638936177159489555194143120408137204708094896402485073482948116954]
dPrimeSig = [62995934878104750991486545310252372051361932832154545640189385031392307270884,16776120925940136440924794112454091454058244627747258413766612398003715874135]

# Public parameters tuple
pubParams = (E, p, G, q)

# Run verification on both messages
valid1 = ECDSAVerify(pubParams, V, d, dSig)
valid2 = ECDSAVerify(pubParams, V, dPrime, dPrimeSig)

# Decode messages
msg1 = intToText(d)
msg2 = intToText(dPrime)

# Ouput results
print("Message 1:", msg1)
print("Signature valid?", valid1)
print()
print("Message 2:", msg2)
print("Signature valid?", valid2)


Message 1: I've never found math hard.
Signature valid? False

Message 2: I find math hard, but cool!
Signature valid? True


In [ ]:
# Testing 
# My message
E = (0, 7)  # Elliptic curve parameters (A, B)
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663  # Prime
G = [55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424]  # Base point
q = 115792089237316195423570985008687907852837564279074904382605163141518161494337  # Order of the base point

pubParams = (E, p, G, q)

# My message
message = "Trust me, Im verified"

# Convert to int
d = textToInt(message) % q

# Generate my ECDSA key
s, V = ECDSAKeyCreation(pubParams)

# Sign it
signature = ECDSASign(pubParams, s, d)

# Verify it
isValid = ECDSAVerify(pubParams, V, d, signature)
print("Signature valid?", isValid)

print("Message:", message)
print("Document D:", d)
print("Signature (s1, s2):", signature)
print("Public verification key V (x, y):", V)

Signature valid? True
Message: Trust me, Im verified
Document D: 146729122300524217703385237801041744736034919641684
Signature (s1, s2): (84893600292549333154071291898678686284890446881808718623013009838375235672504, 12372150166112466802865451329423747454893370195806985666408664117047091378199)
Public verification key V (x, y): (26886000101857175043744223012064840914399359858500524580729933506611517389773, 36435371160224830493988358985383923578454940005123420931665410139142957109858)


In [ ]:
# Teammate verification
E = (0, 7)  # Elliptic curve parameters (A, B)
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663  # Prime
G = [55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424]  # Base point
q = 115792089237316195423570985008687907852837564279074904382605163141518161494337  # Order of the base point

# Teammate verification key V
V = [5469354999477520499808945503844887930789204270483675984714990294151760081120,
     98402707874290859821231649702763060152057496858306507983104049581640518662498]

# Teammate's document (D), signature (S1, S2), and public verification key (V)
d = 3517038505505862061870514343128042740059425830670746877288434638921  # Document (integer form)
s1 = 8859232831611590261127778634489611908135893482648293429253820460052211732410
s2 = 93469223044033998045414999494560080854886050858898810127590916951239653412991

# Convert document to text and read message
message = intToText(d)
print("Decoded mesage:", message)

# Verify the signature
valid = ECDSAVerify((E, p, G, q), V, d, [s1, s2])

# Result
print("Signature valid?", valid)

Decoded mesage: I love skiing but math more!
Signature valid? True


In [ ]:
# Testing
# Parameters for question 8
# Bitcoin curve order q
q = 115792089237316195423570985008687907852837564279074904382605163141518161494337

# Public verification key 
V = [4051293998585674784991639592782214972820158391371785981004352359465450369227,
     88166831356626186178414913298033275054086243781277878360288998796587140930350]

# Documents
d1 = 36762444129640
d2 = 10346484809526143861289411944

# Signatures
s1 = dSig[0]
s2 = dSig[1]
s2_prime = dPrimeSig[1]

# Recover ehpemeral key e
numerator = (d1-d2) % q
denominator = (s2 - s2_prime) % q
den_inv = modInverse(denominator, q) 
e = (numerator * den_inv) % q

# Recover secret signing key s
s1_inv = modInverse(s1, q)
s = ((s2 * e - d1)* s1_inv) % q

print("Recovered ephemeral key e: ", e)
print("Recovered signing key s: ", s)

Recovered ephemeral key e:  31415962
Recovered signing key s:  123456789
